# **Grabbing Show Data from IMDB**

In [126]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [8]:
url = "https://www.imdb.com/title/tt0944947/"
r = requests.get(url)
soup = bs(r.content)

In [20]:
class Episode:
    def __init__(self, title, runtime, rating, season, number):
        self.title = title
        self.runtime = runtime
        self.rating = rating
        self.season = season
        self.number = number
    
    def to_dict(self):
        return {
            'Title': self.title,
            'Runtime': self.runtime,
            'Rating': self.rating,
            'Season': self.season,
            'Number': self.number
        }

In [129]:
seasons = soup.select("div.seasons-and-year-nav div")[2].find_all("a")

show = []

def get_runtime(episode):
    a = episode.select("div.info strong a")[0]['href']
    r = requests.get("https://www.imdb.com" + a)
    soup = bs(r.content)
    return int(pd.to_timedelta(soup.select("time")[0].string.strip()).seconds / 60)

for season in seasons:
    season_r = requests.get("http://imdb.com" + season['href'])
    seasons_page = bs(season_r.content)
    episodes = seasons_page.find("div", {"class": "list detail eplist"}).find_all("div", recursive=False)
    for ep in episodes:
        title = ep.select("div.info strong a")[0].string
        rating = ep.select("div.info div.ipl-rating-widget div.ipl-rating-star span.ipl-rating-star__rating")[0].string
        runtime = get_runtime(ep)
        episode_number = ep.find("meta")['content']
        show.append(Episode(title, runtime, rating, season.text, episode_number))

In [130]:
episodes = pd.DataFrame()
episodes = pd.DataFrame().from_records([e.to_dict() for e in show])
episodes

,Title,Runtime,Rating,Season,Number
0,Winterfell,54,7.5,8,1
1,A Knight of the Seven Kingdoms,58,7.8,8,2
2,The Long Night,82,7.5,8,3
3,The Last of the Starks,78,5.5,8,4
4,The Bells,78,6.0,8,5
...,...,...,...,...,...
68,A Golden Crown,53,9.2,1,6
69,You Win or You Die,58,9.2,1,7
70,The Pointy End,59,9.0,1,8
71,Baelor,57,9.6,1,9


In [132]:
episodes.to_csv('show_data.csv', index=None)